In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')
items = pd.read_csv('train.csv',nrows=1).columns
print(items[36])
#print(df["HiDir"].values)
print(df.shape)

PG
(40000, 37)


In [2]:
X_train = np.zeros((40000,28))
X_test = np.zeros((22446,28))
Y_train =  np.zeros((40000,7))
Y_test = np.zeros((22446,7))
for i in range(2,8):
    temp = df[items[i]].values
    X_train[:,(i-2)]=temp
temp = df[items[8]].values
le.fit(temp)
temp = le.transform(temp) 
X_train[:,6]=temp
for i in range(9,10):
    temp = df[items[i]].values
    X_train[:,(i-2)]=temp
temp = df[items[11]].values
le.fit(temp)
temp = le.transform(temp) 
X_train[:,9]=temp
for i in range(12,30):
    temp = df[items[i]].values
    X_train[:,(i-2)]=temp

In [3]:
for i in range(2,8):
    temp = df2[items[i]].values
    X_test[:,(i-2)]=temp
temp = df2[items[8]].values
le.fit(temp)
temp = le.transform(temp) 
X_test[:,6]=temp
for i in range(9,10):
    temp = df2[items[i]].values
    X_test[:,(i-2)]=temp
temp = df2[items[11]].values
le.fit(temp)
temp = le.transform(temp) 
X_test[:,9]=temp
for i in range(12,30):
    temp = df2[items[i]].values
    X_test[:,(i-2)]=temp

In [4]:
for i in range(30,37):
    temp = df[items[i]].values
    Y_train[:,(i-30)]=temp

In [5]:
W1 = tf.get_variable("W1", [14,28], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
W2 = tf.get_variable("W2", [10,14], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
W3 = tf.get_variable("W3", [7,10], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
b1 = tf.get_variable("b1", [1,14], initializer = tf.zeros_initializer())
b2 = tf.get_variable("b2", [1,10], initializer = tf.zeros_initializer())
b3 = tf.get_variable("b3", [1,7], initializer = tf.zeros_initializer())
parameters = {"W1":W1 , "W2":W2,"W3":W3, "b1":b1, "b2":b2, "b3":b3 }


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    b1 = parameters['b1']
    b2 = parameters['b2']
    b3 = parameters['b3']
    Z1 = tf.add(tf.matmul(X,tf.transpose(W1)),b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(A1,tf.transpose(W2)),b2)
    A2 = tf.nn.relu(Z2)
    Z3 = tf.add(tf.matmul(A2,tf.transpose(W3)),b3)
    return Z3

In [18]:
def rmsletf(y_pred, y_true): 
    y_pred= tf.cast(y_pred, tf.float64) 
    y_true= tf.cast(y_true, tf.float64) 
    y_pred=tf.nn.relu(y_pred) 
    return tf.sqrt(tf.reduce_mean(tf.squared_difference(tf.log1p(y_pred), tf.log1p(y_true))))

In [21]:
XP = tf.placeholder(tf.float32,shape=(40000,28), name="X")
XP1 = tf.placeholder(tf.float32,shape=(22446,28), name="X1")
YP = tf.placeholder(tf.float32,shape=(40000,7), name="Y")
Z3 = forward_propagation(XP,parameters)
print(Y_train.shape)
print(Z3.shape)
cost = rmsletf(Z3,Y_train)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.003).minimize(cost)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(2200):
    _,min_cost = sess.run([optimizer,cost] ,feed_dict={XP : X_train, YP: Y_train})
    if(i%100==0):
        print(min_cost)

(40000, 7)
(40000, 7)
3.2415226487237256
2.866300471999093
2.855134461590561
2.269310008372061
2.2190281007346466
2.2068801422613964
2.1982501952563513
2.192018469814168
2.1871274740130935
2.1840251233162142
2.182261414906378
2.1793202276940153
2.178066757499201
2.177309853419186
2.1769459918035228
2.1764585250800086
2.1762591702226497
2.176045346388458
2.1758766367082716
2.175677989987062
2.1756544323996994
2.175404753415933


In [24]:
Z4 = forward_propagation(XP1,parameters)
Z4 = sess.run(Z4,feed_dict={XP1 : X_test} )
tem1 = df2['ID'].values
mycsv = open("output.csv", "w")
columnTitleRow = "ID,PA,PB,PC,PD,PE,PF,PG\n"
mycsv.write(columnTitleRow)
for i in range(22446):
    row = str(tem1[i]) + "," + str(Z4[i][0]) + "," + str(Z4[i][1])+ "," + str(Z4[i][2]) + "," + str(Z4[i][3]) + "," + str(Z4[i][4]) + "," + str(Z4[i][5]) + "," + str(Z4[i][6]) + "\n"
    mycsv.write(row)

In [23]:
print(np.where(np.isnan(Z4)))

(array([], dtype=int64), array([], dtype=int64))
